# Análise Exploratória e Pré-Processamento de Dados para Machine Learning

### Autor: 
- [Thomas Chiari Ciocchetti de Souza](https://github.com/thomaschiari)

Grande parte de um projeto de Machine Learning se trata de entender o problema, avaliar os dados e prepará-los para o modelo. Neste notebook, vamos explorar algumas técnicas de análise exploratória e pré-processamento de dados.

Para um dataset, podemos ter diversos tipos de variáveis, como numéricas, categóricas, ordinais, binárias, etc. Cada tipo de variável requer um tratamento diferente. Além disso, podemos ter dados faltantes, outliers, variáveis altamente correlacionadas, etc.

Portanto, nessa etapa, é necessário: 
- Entender o problema
- Entender os tipos de dados
- Entender os dados faltantes e como tratá-los
- Compreender a natureza das variáveis categoricas e como tratá-las
- Feature Engineering
- Análise estatística dos dados

## Parte 1: Análise Exploratória de Dados

Para a análise exploratória, utilizaremos o dataset [House Prices: Advanced Regression Techniques](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data), disponível no Kaggle. O objetivo é prever o preço de venda de casas em Ames, Iowa, EUA, com base em diversas variáveis explicativas. Se trata de um problema de regressão, visto que estamos buscando prever um valor contínuo.

In [17]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

In [18]:
import os
PATH = os.path.join('data', 'house-prices', 'train.csv')

In [19]:
dataset = pd.read_csv(PATH, encoding='utf-8')
dataset.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


Nesse dataset, temos 79 variáveis explicativas, que podem ser numéricas ou categóricas, e uma variável resposta, além de uma variável identificadora. A variável resposta é, obviamente, a que desejamos prever, e a variável identificadora é apenas um identificador único para cada casa. É possível que essa variável não seja útil para o modelo, mas, por enquanto, vamos mantê-la. é possível encontrar a descrição de cada variável no arquivo [`data_description.txt`](data/house-prices/data_description.txt).

Vamos iniciar com uma descrição geral do dataset, obtendo algumas informações básicas sobre as variáveis.

In [20]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [21]:
dataset.describe().T

,count,mean,std,min,25%,50%,75%,max
Id,1460.0,730.500000,421.610009,1.0,365.75,730.5,1095.25,1460.0
MSSubClass,1460.0,56.897260,42.300571,20.0,20.00,50.0,70.00,190.0
LotFrontage,1201.0,70.049958,24.284752,21.0,59.00,69.0,80.00,313.0
LotArea,1460.0,10516.828082,9981.264932,1300.0,7553.50,9478.5,11601.50,215245.0
OverallQual,1460.0,6.099315,1.382997,1.0,5.00,6.0,7.00,10.0
OverallCond,1460.0,5.575342,1.112799,1.0,5.00,5.0,6.00,9.0
YearBuilt,1460.0,1971.267808,30.202904,1872.0,1954.00,1973.0,2000.00,2010.0
YearRemodAdd,1460.0,1984.865753,20.645407,1950.0,1967.00,1994.0,2004.00,2010.0
MasVnrArea,1452.0,103.685262,181.066207,0.0,0.00,0.0,166.00,1600.0
BsmtFinSF1,1460.0,443.639726,456.098091,0.0,0.00,383.5,712.25,5644.0


In [22]:
df = dataset.copy()

# Colunas com valores quantitativos
QUANTITATIVE_COLS = ['LotFrontage', 'LotArea', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
                    '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'GarageYrBlt', 'GarageArea', 'BsmtFullBath', 'BsmtHalfBath', 
                    'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces','GarageCars',  'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch',
                    '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal']

# Colunas com valores ordinais
ORDINAL_COLS = ['Utilities', 'LandSlope', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'ExterQual', 'ExterCond',
              'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'HeatingQC', 'KitchenQual',
              'FireplaceQu', 'GarageYrBlt', 'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'YrSold', 'MoSold']

# Colunas com valores nominais
NOMINAL_COLS = list(set(df.columns) - set(QUANTITATIVE_COLS + ORDINAL_COLS + ['SalePrice']))


Após analisar a documentação do dataset, pudemos separar as variáveis em 3 grupos:
- Variáveis numéricas
- Variáveis categóricas
- Variáveis ordinais

